In [49]:
from pandas import DataFrame
from pandas import read_csv
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler

dataset = read_csv('LOAD.csv', header=0, index_col=0).iloc[::-1]

In [50]:
dataset[dataset.columns] = StandardScaler().fit_transform(dataset)

In [51]:
dataset_shifted = dataset.shift(-5)
dataset = dataset.iloc[:-5]
dataset_shifted = dataset_shifted.iloc[:-5]

In [52]:
train_dataset = dataset.iloc[:-500]
train_dataset_shifted = dataset_shifted.iloc[:-500]

test_dataset = dataset.iloc[-500:]
test_dataset_shifted = dataset_shifted.iloc[-500:]

In [ ]:
#Exhaustive search
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import copy

def fit_lstm_tuning(n_test = 28, n_lag = 1, n_batch = 1, n_neurons = 1, activation = 'sigmoid', optimizer = 'adam'):
    model_lstm = Sequential()
    model_lstm.add(LSTM(n_neurons, input_shape=(n_batch, dataset.shape[1]),activation = activation))
    model_lstm.add(Dense(1))
    model_lstm.compile(loss='mae', optimizer=optimizer)
    return model_lstm

n_lag = [1]
n_seq = [1]
n_batch = [1]
n_neurons = [1,2,3]
activation = ['softmax', 'relu', 'tanh', 'sigmoid']
optimizer = ['SGD', 'RMSprop','Adam']

X= copy.deepcopy(train_dataset.values.reshape((train_dataset.shape[0], 1, train_dataset.shape[1])))
Y = copy.deepcopy(train_dataset_shifted['system.load.5'])

# create model
model = KerasClassifier(build_fn=fit_lstm_tuning, epochs = 1500, batch_size = 1, verbose=0)

param_grid = dict(n_batch = n_batch, n_neurons = n_neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X,Y)

# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(10, input_shape=(1, dataset.shape[1])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')

model_lstm.fit(train_dataset.values.reshape((train_dataset.shape[0], 1, train_dataset.shape[1])), train_dataset_shifted['system.load.5'], verbose=2, shuffle=False)

In [ ]:
test_dataset_predicted = model_lstm.predict(test_dataset.values.reshape((test_dataset.shape[0], 1, test_dataset.shape[1])))

In [ ]:
pyplot.plot(test_dataset_shifted['system.load.5'], label='real')
pyplot.plot(test_dataset_predicted, label='lstm')
pyplot.legend()
pyplot.show()

In [ ]:
mean_absolute_error(test_dataset_predicted, test_dataset_shifted['system.load.5'])